# Librerías a usar

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.language_models.llms import LLM
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import Field
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from typing import Optional, List, Any
from langchain.schema import Document
from transformers import pipeline
from peft import PeftModel
import gradio as gr
import pandas as pd
import gradio as gr
import numpy as np
import json
import torch
import gc
import re
import os

d:\Python_Envs\torch311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python_Envs\torch311\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Generacion de archivos para alimentar el sistema RAG

In [2]:
df = pd.read_excel('baseline.xlsx')
df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
df.columns

Index(['tramp_id', 'sampling_date', 'lat', 'lon', 'municipality',
       'square_area', 'plantation_age', 'capture_count', 'state',
       'square_area_imputed', 'month', 'year', 'year_month', 'day_of_year_sin',
       'day_of_year_cos', 'day_of_week_sin', 'day_of_week_cos',
       'week_of_year_sin', 'week_of_year_cos', 'month_sin', 'month_cos',
       'critical_season', 'severity_encoded', 'distance_to_nearest_hotspot',
       'hotspots_within_5km', 'text_feature_location', 'text_feature_risk',
       'text_feature_capture', 'text_feature_plantation',
       'text_feature_all_things'],
      dtype='object')

In [3]:
df_backup = df.copy()

In [8]:
df = df_backup
len(df)

827856

In [11]:
df.dtypes

tramp_id                               object
sampling_date                  datetime64[ns]
lat                                   float64
lon                                   float64
municipality                           object
square_area                           float64
plantation_age                          int64
capture_count                         float64
state                                  object
square_area_imputed                   float64
month                                   int64
year                                    int64
year_month                             object
day_of_year_sin                       float64
day_of_year_cos                       float64
day_of_week_sin                       float64
day_of_week_cos                       float64
week_of_year_sin                      float64
week_of_year_cos                      float64
month_sin                             float64
month_cos                             float64
critical_season                   

In [12]:
year_mask = df['year'] >= 2024
month_mask = df['month'] >= 1
df = df[year_mask]
df = df[month_mask] 
len(df)

158408

In [ ]:
severity_dict = {
    0: 'sin riesgo',
    1: 'de riesgo leve',
    2: 'de riesgo moderado',
    3: 'de riesgo severo'
}

critical_season_dict = {
    0: 'normal',
    1: 'critica'
}


area_information_dict = {
    'original': " (area historica registrada correctamente)", 
    'radius_regressor' : "(el area en hectareas se calculo usando trampas cercanas)", 
    'median' : "(el area en hectareas se calculo usando la mediana de los datos de muestra)", 
    'same_trap_id_temporal' : " (valor del area en hectarea obtenido usando registros historicos)"
}

### Generación de archivos para fine-tuning de Phi3

In [ ]:
def build_rich_input(x):
    return (
        f"tramp_id: {x['tramp_id']}, "
        f"sampling_date: {x['sampling_date'].strftime('%d-%m-%Y')}, "
        f"lat: {x['lat']}, lon: {x['lon']}, "
        f"municipality: {x['municipality']}, state: {x['state']}, "
        f"capture_count: {x['capture_count']}, severity: {x['severity_encoded']}, "
        f"critical_season: {x['critical_season']}, "
        f"hotspots_within_5km: {x['hotspots_within_5km']}, "
        f"distance_to_nearest_hotspot: {x['distance_to_nearest_hotspot']}, "
        f"plantation_age: {x['plantation_age']}, "
        f"square_area_imputed: {x['square_area_imputed']}, "
        f"month: {x['month']}, year_month: {x['year_month']}, "
        f"day_of_year_sin: {x['day_of_year_sin']}, day_of_year_cos: {x['day_of_year_cos']}, "
        f"week_of_year_sin: {x['week_of_year_sin']}, week_of_year_cos: {x['week_of_year_cos']}, "
    )

df["input_rich"] = df.apply(build_rich_input, axis=1)

In [ ]:
def generate_output_risk_temporal(x):
    cc = x['capture_count']
    cs = x['critical_season']
    sev = x['severity_encoded']

    if cc > 15 and cs == 1:
        return "El conteo de capturas es alto durante temporada crítica. Esto sugiere un riesgo elevado consistente con los patrones estacionales recientes."
    elif cc > 0 and cs == 1:
        return "Hay capturas moderadas en temporada crítica. El comportamiento es esperado, sin indicios de anomalía fuerte."
    elif sev >= 2:
        return "Aunque no es temporada crítica, la severidad es moderada. Conviene mantener vigilancia."
    else:
        return "El nivel de captura es bajo y fuera de temporada crítica. Riesgo estable sin cambios relevantes."
    
def generate_output_geospatial(x):
    h5 = x['hotspots_within_5km']
    dist = x['distance_to_nearest_hotspot']
    sev = x['severity_encoded']

    if h5 >= 3 and dist < 1500:
        return "La trampa se ubica cerca de varios hotspots activos. La presión fitosanitaria es alta y el riesgo es creciente."
    elif h5 >= 1 and dist < 3000:
        return "Hay hotspots cercanos con distancia moderada. Existe riesgo medio que requiere seguimiento."
    elif sev >= 2:
        return "Sin hotspots cercanos, pero la severidad es moderada. Riesgo localizado que debe vigilarse."
    else:
        return "Sin hotspots significativos y severidad baja. Riesgo estable en la zona."


def generate_output_vulnerability(x):
    age = x['plantation_age']
    area = x['square_area_imputed']
    cc = x['capture_count']

    if age >= 5 and cc > 10:
        return "La plantación madura presenta capturas elevadas. Existe vulnerabilidad alta a infestación."
    elif age >= 3 and cc > 0:
        return "Plantación de edad media con capturas moderadas. Riesgo potencial que debe monitorearse."
    elif area > 5 and cc > 0:
        return "Predio de gran superficie con presencia de capturas. El tamaño puede facilitar propagación, riesgo medio."
    else:
        return "La combinación de edad, superficie y capturas no sugiere vulnerabilidad relevante."


def generate_output_weekly_anomaly(x):
    cc = x['capture_count']
    sev = x['severity_encoded']
    week_sin = x['week_of_year_sin']
    week_cos = x['week_of_year_cos']

    week_mag = (week_sin**2 + week_cos**2)**0.5

    if cc > 20 and week_mag > 0.9:
        return "La actividad de capturas es mayor al patrón típico de la semana. Posible anomalía detectada."
    elif cc > 5:
        return "Las capturas son moderadas para esta semana. El comportamiento es cercano al patrón esperado."
    else:
        return "Actividad baja y dentro del rango esperado para la semana."


def generate_output_municipal_risk(x):
    h5 = x['hotspots_within_5km']
    cc = x['capture_count']
    sev = x['severity_encoded']
    muni = x['municipality']

    if h5 >= 3 and cc > 10:
        return f"En {muni}, los datos recientes muestran capturas altas y varios hotspots cercanos. El riesgo municipal es elevado."
    elif h5 >= 1 and cc > 5:
        return f"En {muni}, hay presión moderada por hotspots y capturas. Riesgo municipal medio."
    elif sev >= 2:
        return f"En {muni}, la severidad local es moderada pese a baja actividad geográfica. Riesgo localizado."
    else:
        return f"En {muni}, el comportamiento reciente es estable. No hay señales de riesgo destacado."


df["output_risk_temporal"] = df.apply(generate_output_risk_temporal, axis=1)
df["output_geospatial"] = df.apply(generate_output_geospatial, axis=1)
df["output_vulnerability"] = df.apply(generate_output_vulnerability, axis=1)
df["output_weekly_anomaly"] = df.apply(generate_output_weekly_anomaly, axis=1)
df["output_municipal_risk"] = df.apply(generate_output_municipal_risk, axis=1)

instructions_temporal = [
    "Evalúa el riesgo fitosanitario usando temporada crítica, capturas y estacionalidad. Usa solo datos de julio 2025 en adelante.",
    "Determina si el comportamiento temporal del gorgojo es normal o anómalo usando month y critical_season."
]

instructions_geospatial = [
    "Evalúa el riesgo usando hotspots cercanos, distancia y severidad. Habla solo basado en datos posteriores a julio 2025.",
    "Determina si hay presión fitosanitaria cercana según hotspots_within_5km y distancia al hotspot."
]

instructions_vulnerability = [
    "Evalúa la vulnerabilidad combinando edad de plantación, superficie y capturas usando datos posteriores a julio 2025.",
    "Determina si la plantación es propensa a infestación según plantation_age, area y capturas."
]

instructions_weekly = [
    "Determina si existe anomalía semanal usando capture_count y las variables week_of_year_sin/cos desde julio 2025.",
    "Compara la actividad semanal con el patrón esperado para identificar anomalías."
]

instructions_municipal = [
    "Resume el riesgo municipal usando capturas recientes, hotspots y year_month solo con datos desde julio 2025.",
    "Evalúa si el municipio muestra tendencia de riesgo según capturas y hotspots cercanos."
]

def generate_jsonl(path, df, instruction_list, output_col):
    with open(path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            for inst in instruction_list:
                record = {
                    "instruction": inst,
                    "input": row["input_rich"],
                    "output": row[output_col]
                }
                f.write(json.dumps(record, ensure_ascii=False) + "\n")
    print(f"✅ JSONL generado: {path}")
    
    
generate_jsonl("agave_risk_temporal.jsonl", df, instructions_temporal, "output_risk_temporal")
generate_jsonl("agave_geospatial.jsonl", df, instructions_geospatial, "output_geospatial")
generate_jsonl("agave_vulnerability.jsonl", df, instructions_vulnerability, "output_vulnerability")
generate_jsonl("agave_weekly_anomaly.jsonl", df, instructions_weekly, "output_weekly_anomaly")
generate_jsonl("agave_municipal_risk.jsonl", df, instructions_municipal, "output_municipal_risk")

# Generación de archivos para generar embeddings

In [15]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

def generate_natural_language_description(record):
    
    agg_level = record['aggregation_level']
    
    # Construir la descripción según el nivel de agregación
    if agg_level == 'state':
        location = f"en el estado de {record['state']}"
        
    elif agg_level == 'municipality':
        location = f"en el municipio de {record['municipality']}, {record['state']}"
        
    elif agg_level == 'month':
        location = f"a nivel nacional durante el mes de {record['month']}"
        
    elif agg_level == 'state_month':
        location = f"en el estado de {record['state']} durante el mes de {record['month']}"
        
    elif agg_level == 'municipality_month':
        location = f"en el municipio de {record['municipality']}, {record['state']}, durante el mes de {record['month']}"
    
    # Construcción de la descripción en lenguaje natural
    description_parts = []
    
    # Introducción con contexto geográfico/temporal
    intro = f"En el período de {record['year_range']}, {location},"
    description_parts.append(intro)
    
    # Estadísticas de captura
    capture_desc = (
        f"se registró un promedio de {record['avg_captures']:.2f} gorgojos picudos capturados por trampa. "
        f"La mediana de capturas fue de {record['median_captures']:.2f}, "
        f"con un rango intercuartílico entre {record['p25_captures']:.2f} (percentil 25) "
        f"y {record['p75_captures']:.2f} (percentil 75)."
    )
    description_parts.append(capture_desc)
    
    # Variabilidad
    if record['std_captures'] is not None:
        variability_desc = f"La desviación estándar de las capturas fue de {record['std_captures']:.2f}, "
        
        # Interpretar la variabilidad
        cv = (record['std_captures'] / record['avg_captures']) * 100 if record['avg_captures'] > 0 else 0
        if cv < 50:
            variability_desc += "lo que indica una variabilidad relativamente baja en las capturas."
        elif cv < 100:
            variability_desc += "lo que indica una variabilidad moderada en las capturas."
        else:
            variability_desc += "lo que indica una alta variabilidad en las capturas."
        
        description_parts.append(variability_desc)
    
    # Información de muestreo
    sample_desc = f"Estos datos se basan en {record['observation_count']} observaciones registradas."
    description_parts.append(sample_desc)
    
    # Edad de plantación
    if record['avg_plantation_age'] is not None:
        plantation_desc = (
            f"La edad promedio de las plantaciones de agave en esta área fue de "
            f"{record['avg_plantation_age']:.2f} años."
        )
        description_parts.append(plantation_desc)
    
    # Interpretación de nivel de riesgo basada en capturas promedio
    risk_level = ""
    if record['avg_captures'] < 5:
        risk_level = "bajo nivel de infestación"
    elif record['avg_captures'] < 15:
        risk_level = "nivel moderado de infestación"
    elif record['avg_captures'] < 30:
        risk_level = "nivel alto de infestación"
    else:
        risk_level = "nivel crítico de infestación"
    
    risk_desc = f"Este promedio de capturas sugiere un {risk_level} del picudo del agave en la zona."
    description_parts.append(risk_desc)
    
    # Unir todas las partes
    full_description = " ".join(description_parts)
    
    return full_description


def generate_rag_jsonl(df, output_file):
    """
    Genera un archivo JSONL optimizado para sistemas RAG con descripciones
    en lenguaje natural de las estadísticas de capturas de picudo.
    
    Parameters:
    -----------
    input_file : str
        Ruta al archivo Excel con los datos
    output_file : str
        Ruta donde se guardará el archivo JSONL para RAG
    """
      
    # Extraer nombre del mes (month ya existe como int, solo necesitamos el nombre)
    df['month_name'] = df['sampling_date'].dt.month_name()
    
    # Lista para almacenar todos los registros RAG
    rag_records = []
    
    # 1. Agregación por ESTADO
    print("Procesando agregaciones por Estado...")
    state_agg = df.groupby('state').agg({
        'capture_count': ['mean', 'std', 'count', 'median', 
                         lambda x: np.percentile(x, 25),
                         lambda x: np.percentile(x, 75)],
        'year': ['min', 'max'],
        'plantation_age': 'mean'
    }).reset_index()
    
    state_agg.columns = ['state', 'avg_captures', 'std_captures', 'observation_count',
                        'median_captures', 'p25_captures', 'p75_captures',
                        'year_min', 'year_max', 'avg_plantation_age']
    
    for _, row in state_agg.iterrows():
        stats = {
            'aggregation_level': 'state',
            'state': row['state'],
            'municipality': None,
            'month': None,
            'avg_captures': round(float(row['avg_captures']), 2),
            'std_captures': round(float(row['std_captures']), 2) if pd.notna(row['std_captures']) else None,
            'median_captures': round(float(row['median_captures']), 2),
            'p25_captures': round(float(row['p25_captures']), 2),
            'p75_captures': round(float(row['p75_captures']), 2),
            'observation_count': int(row['observation_count']),
            'year_range': f"{int(row['year_min'])}-{int(row['year_max'])}",
            'avg_plantation_age': round(float(row['avg_plantation_age']), 2) if pd.notna(row['avg_plantation_age']) else None
        }
        
        # Generar descripción en lenguaje natural
        natural_description = generate_natural_language_description(stats)
        
        # Crear registro RAG
        rag_record = {
            'text': natural_description,
            'metadata': {
                'aggregation_level': stats['aggregation_level'],
                'state': stats['state'],
                'municipality': stats['municipality'],
                'month': stats['month'],
                'year_range': stats['year_range'],
                'avg_captures': stats['avg_captures'],
                'observation_count': stats['observation_count']
            }
        }
        rag_records.append(rag_record)
    
    # 2. Agregación por MUNICIPIO
    print("Procesando agregaciones por Municipio...")
    muni_agg = df.groupby(['state', 'municipality']).agg({
        'capture_count': ['mean', 'std', 'count', 'median',
                         lambda x: np.percentile(x, 25),
                         lambda x: np.percentile(x, 75)],
        'year': ['min', 'max'],
        'plantation_age': 'mean'
    }).reset_index()
    
    muni_agg.columns = ['state', 'municipality', 'avg_captures', 'std_captures', 
                        'observation_count', 'median_captures', 'p25_captures', 
                        'p75_captures', 'year_min', 'year_max', 'avg_plantation_age']
    
    for _, row in muni_agg.iterrows():
        stats = {
            'aggregation_level': 'municipality',
            'state': row['state'],
            'municipality': row['municipality'],
            'month': None,
            'avg_captures': round(float(row['avg_captures']), 2),
            'std_captures': round(float(row['std_captures']), 2) if pd.notna(row['std_captures']) else None,
            'median_captures': round(float(row['median_captures']), 2),
            'p25_captures': round(float(row['p25_captures']), 2),
            'p75_captures': round(float(row['p75_captures']), 2),
            'observation_count': int(row['observation_count']),
            'year_range': f"{int(row['year_min'])}-{int(row['year_max'])}",
            'avg_plantation_age': round(float(row['avg_plantation_age']), 2) if pd.notna(row['avg_plantation_age']) else None
        }
        
        natural_description = generate_natural_language_description(stats)
        
        rag_record = {
            'text': natural_description,
            'metadata': {
                'aggregation_level': stats['aggregation_level'],
                'state': stats['state'],
                'municipality': stats['municipality'],
                'month': stats['month'],
                'year_range': stats['year_range'],
                'avg_captures': stats['avg_captures'],
                'observation_count': stats['observation_count']
            }
        }
        rag_records.append(rag_record)
    
    # 3. Agregación por MES
    print("Procesando agregaciones por Mes...")
    month_agg = df.groupby(['month', 'month_name']).agg({
        'capture_count': ['mean', 'std', 'count', 'median',
                         lambda x: np.percentile(x, 25),
                         lambda x: np.percentile(x, 75)],
        'year': ['min', 'max'],
        'plantation_age': 'mean'
    }).reset_index()
    
    month_agg.columns = ['month_num', 'month_name', 'avg_captures', 'std_captures',
                        'observation_count', 'median_captures', 'p25_captures',
                        'p75_captures', 'year_min', 'year_max', 'avg_plantation_age']
    
    month_agg = month_agg.sort_values('month_num')
    
    for _, row in month_agg.iterrows():
        stats = {
            'aggregation_level': 'month',
            'state': None,
            'municipality': None,
            'month': row['month_name'],
            'avg_captures': round(float(row['avg_captures']), 2),
            'std_captures': round(float(row['std_captures']), 2) if pd.notna(row['std_captures']) else None,
            'median_captures': round(float(row['median_captures']), 2),
            'p25_captures': round(float(row['p25_captures']), 2),
            'p75_captures': round(float(row['p75_captures']), 2),
            'observation_count': int(row['observation_count']),
            'year_range': f"{int(row['year_min'])}-{int(row['year_max'])}",
            'avg_plantation_age': round(float(row['avg_plantation_age']), 2) if pd.notna(row['avg_plantation_age']) else None
        }
        
        natural_description = generate_natural_language_description(stats)
        
        rag_record = {
            'text': natural_description,
            'metadata': {
                'aggregation_level': stats['aggregation_level'],
                'state': stats['state'],
                'municipality': stats['municipality'],
                'month': stats['month'],
                'year_range': stats['year_range'],
                'avg_captures': stats['avg_captures'],
                'observation_count': stats['observation_count']
            }
        }
        rag_records.append(rag_record)
    
    # 4. Agregación por ESTADO + MES
    print("Procesando agregaciones por Estado + Mes...")
    state_month_agg = df.groupby(['state', 'month', 'month_name']).agg({
        'capture_count': ['mean', 'std', 'count', 'median',
                         lambda x: np.percentile(x, 25),
                         lambda x: np.percentile(x, 75)],
        'year': ['min', 'max'],
        'plantation_age': 'mean'
    }).reset_index()
    
    state_month_agg.columns = ['state', 'month_num', 'month_name', 'avg_captures',
                               'std_captures', 'observation_count', 'median_captures',
                               'p25_captures', 'p75_captures', 'year_min', 'year_max',
                               'avg_plantation_age']
    
    for _, row in state_month_agg.iterrows():
        stats = {
            'aggregation_level': 'state_month',
            'state': row['state'],
            'municipality': None,
            'month': row['month_name'],
            'avg_captures': round(float(row['avg_captures']), 2),
            'std_captures': round(float(row['std_captures']), 2) if pd.notna(row['std_captures']) else None,
            'median_captures': round(float(row['median_captures']), 2),
            'p25_captures': round(float(row['p25_captures']), 2),
            'p75_captures': round(float(row['p75_captures']), 2),
            'observation_count': int(row['observation_count']),
            'year_range': f"{int(row['year_min'])}-{int(row['year_max'])}",
            'avg_plantation_age': round(float(row['avg_plantation_age']), 2) if pd.notna(row['avg_plantation_age']) else None
        }
        
        natural_description = generate_natural_language_description(stats)
        
        rag_record = {
            'text': natural_description,
            'metadata': {
                'aggregation_level': stats['aggregation_level'],
                'state': stats['state'],
                'municipality': stats['municipality'],
                'month': stats['month'],
                'year_range': stats['year_range'],
                'avg_captures': stats['avg_captures'],
                'observation_count': stats['observation_count']
            }
        }
        rag_records.append(rag_record)
    
    # 5. Agregación por MUNICIPIO + MES
    print("Procesando agregaciones por Municipio + Mes...")
    muni_month_agg = df.groupby(['state', 'municipality', 'month', 'month_name']).agg({
        'capture_count': ['mean', 'std', 'count', 'median',
                         lambda x: np.percentile(x, 25),
                         lambda x: np.percentile(x, 75)],
        'year': ['min', 'max'],
        'plantation_age': 'mean'
    }).reset_index()
    
    muni_month_agg.columns = ['state', 'municipality', 'month_num', 'month_name',
                              'avg_captures', 'std_captures', 'observation_count',
                              'median_captures', 'p25_captures', 'p75_captures',
                              'year_min', 'year_max', 'avg_plantation_age']
    
    for _, row in muni_month_agg.iterrows():
        stats = {
            'aggregation_level': 'municipality_month',
            'state': row['state'],
            'municipality': row['municipality'],
            'month': row['month_name'],
            'avg_captures': round(float(row['avg_captures']), 2),
            'std_captures': round(float(row['std_captures']), 2) if pd.notna(row['std_captures']) else None,
            'median_captures': round(float(row['median_captures']), 2),
            'p25_captures': round(float(row['p25_captures']), 2),
            'p75_captures': round(float(row['p75_captures']), 2),
            'observation_count': int(row['observation_count']),
            'year_range': f"{int(row['year_min'])}-{int(row['year_max'])}",
            'avg_plantation_age': round(float(row['avg_plantation_age']), 2) if pd.notna(row['avg_plantation_age']) else None
        }
        
        natural_description = generate_natural_language_description(stats)
        
        rag_record = {
            'text': natural_description,
            'metadata': {
                'aggregation_level': stats['aggregation_level'],
                'state': stats['state'],
                'municipality': stats['municipality'],
                'month': stats['month'],
                'year_range': stats['year_range'],
                'avg_captures': stats['avg_captures'],
                'observation_count': stats['observation_count']
            }
        }
        rag_records.append(rag_record)
    
    # Guardar todo en un archivo JSONL
    print(f"\nGuardando {len(rag_records)} registros en {output_file}...")
    with open(output_file, 'w', encoding='utf-8') as f:
        for record in rag_records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    print(f"✓ Archivo JSONL para RAG generado exitosamente: {output_file}")
    print(f"\nResumen de registros generados:")
    print(f"  - Por estado: {len(state_agg)} registros")
    print(f"  - Por municipio: {len(muni_agg)} registros")
    print(f"  - Por mes: {len(month_agg)} registros")
    print(f"  - Por estado + mes: {len(state_month_agg)} registros")
    print(f"  - Por municipio + mes: {len(muni_month_agg)} registros")
    print(f"  - TOTAL: {len(rag_records)} registros")
    
    return rag_records


if __name__ == "__main__":
    # Configurar rutas de archivos
    input_file = "tu_archivo_datos.xlsx"  # Cambia esto por tu archivo
    output_file = "capturas_picudo_rag.jsonl"
    
    # Generar el archivo JSONL para RAG
    records = generate_rag_jsonl(df, "historic_data.jsonl")
    
    # Mostrar algunos ejemplos
    print("\n" + "="*80)
    print("EJEMPLOS DE DESCRIPCIONES EN LENGUAJE NATURAL:")
    print("="*80)
    
    # Ejemplo de cada tipo de agregación
    aggregation_types = ['state', 'municipality', 'month', 'state_month', 'municipality_month']
    
    for agg_type in aggregation_types:
        example = next((r for r in records if r['metadata']['aggregation_level'] == agg_type), None)
        if example:
            print(f"\n{agg_type.upper().replace('_', ' ')}:")
            print("-" * 80)
            print(f"TEXTO: {example['text']}")
            print(f"\nMETADATA: {json.dumps(example['metadata'], indent=2, ensure_ascii=False)}")

Procesando agregaciones por Estado...
Procesando agregaciones por Municipio...
Procesando agregaciones por Mes...
Procesando agregaciones por Estado + Mes...
Procesando agregaciones por Municipio + Mes...

Guardando 1553 registros en historic_data.jsonl...
✓ Archivo JSONL para RAG generado exitosamente: historic_data.jsonl

Resumen de registros generados:
  - Por estado: 4 registros
  - Por municipio: 143 registros
  - Por mes: 12 registros
  - Por estado + mes: 48 registros
  - Por municipio + mes: 1346 registros
  - TOTAL: 1553 registros

EJEMPLOS DE DESCRIPCIONES EN LENGUAJE NATURAL:

STATE:
--------------------------------------------------------------------------------
TEXTO: En el período de 2024-2025, en el estado de GUANAJUATO, se registró un promedio de 4.05 gorgojos picudos capturados por trampa. La mediana de capturas fue de 3.00, con un rango intercuartílico entre 1.00 (percentil 25) y 5.00 (percentil 75). La desviación estándar de las capturas fue de 4.70, lo que indica un

# Embeddings nuevos

### Manual operativo modificado

In [2]:
# Cargamos el Manual Operativo como un documento para consulta del bot.
loader = PyPDFLoader(r'C:/Users/Delbert/Documents/Maestria/Proyecto Integrador\Avance 1/Tecnologico-Monterrey-Proyecto-Integrador-equipo-36/Telegram-Bot/Docs for embeddings/ManualOperativo_Agave_Modificado.pdf')
docs = loader.load()

print(f"Se cargaron {len(docs)} paginas desde el Manual Operativo.")

Se cargaron 18 paginas desde el Manual Operativo.


In [3]:
# Cargamos todos los text_features previamente generados que podrán ser consultados de nuevo.

from pathlib import Path
import json

def load_jsonl_as_documents(jsonl_paths):

    all_docs = []

    for file_path in jsonl_paths:
        path = Path(file_path)
        if not path.exists():
            print(f"No se encontró el archivo siguiente: {file_path}")
            continue

        with open(path, "r", encoding="utf-8") as f:
            for line_num, line in enumerate(f, start=1):
                try:
                    record = json.loads(line.strip())
                    # Usamos de nuevo la estructura de Alpaca
                    text = (
                        f"Instrucción: {record.get('instruction', '')}\n"
                        f"Entrada: {record.get('input', '')}\n"
                        f"Respuesta: {record.get('output', '')}"
                    )

                    # Tomamos metadata
                    doc = Document(
                        page_content=text,
                        metadata={
                            "source": path.name,
                            "line": line_num
                        }
                    )
                    all_docs.append(doc)
                except json.JSONDecodeError as e:
                    print(f"Se dio un error en la linea {line_num} en {path.name}: {e}")
                    continue

    print(f"Se cargaron {len(all_docs)} documentos de {len(jsonl_paths)} archivos JSONL.")
    return all_docs

jsonl_files = [
    "Docs for embeddings/agave_allThings.jsonl",
    "Docs for embeddings/agave_capture.jsonl",
    "Docs for embeddings/agave_location.jsonl",
    "Docs for embeddings/agave_plantation.jsonl",
    "Docs for embeddings/agave_risk.jsonl",
    "Docs for embeddings/agave_geospatial.jsonl",
    "Docs for embeddings/agave_municipal_risk.jsonl",
    "Docs for embeddings/agave_risk_temporal.jsonl",
    "Docs for embeddings/agave_vulnerability.jsonl",
    "Docs for embeddings/agave_weekly_anomaly.jsonl",
    "Docs for embeddings/historic_data.jsonl"
]

docs_jsonl = load_jsonl_as_documents(jsonl_files)

Se cargaron 303743 documentos de 11 archivos JSONL.


In [4]:
all_documents = docs + docs_jsonl

In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, 
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    length_function=len,
    is_separator_regex=False,
)

chunked_docs = splitter.split_documents(all_documents)

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code': True
    },
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 12
    }
)

C:\Users\Delbert\AppData\Local\Temp\ipykernel_10108\605335807.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [6]:
vectorstore = FAISS.from_documents(chunked_docs, embedding=embeddings)

# Guardamos la vector store para cargas mas rapidas despues
vectorstore.save_local("rag_faiss_store")